# import

In [1]:

import pandas as pd
from tqdm.notebook import tqdm as tqdm
import time
from graphviz import *
import lightgbm as lgb
from hyperopt import hp, tpe, Trials, fmin,STATUS_OK
import fasttext as ft
# import importlib
# importlib.reload(hoge)
from my_library.horse import *
import pickle


# path

In [2]:
path_ubu = '/home/hipro/デスクトップ/Horse/Data/20_21'
path_win2 = '/Users/rince/Desktop/Horse/Data/saishin2/'
path_win = '/Users/rince/Desktop/Horse/Data/saishin/'
path_win = '/Users/Owner/Desktop/program/Horse/Data/saishin/'
path_win2 = '/Users/Owner/Desktop/program/Horse/Data/saishin2/'
path_ft = '/Users/Owner/Desktop/Horse/ft_data/peds_ft.txt'

# load data

In [3]:
results = load_csv(path_win+'results.csv')
horse_results = load_csv(path_win+'horse_results.csv')
peds = load_csv(path_win+'peds.csv')
# 何回やってもロードすると, nanが出る
peds.fillna('nan',inplace=True)
return_tables = load_csv(path_win+'return.csv')
return_tables.rename(columns={'0':0,'1':1,'2':2,'3':3},inplace=True)

# race_id 命名規則

race_id 202105040802\
yyyy_pp_xx_xxrr\
y : year\
p : palce\
x : 謎\
r : race番号

In [4]:
ll = LearnLGBM(peds,results,horse_results)
ll.path_ft = '/Users/Owner/Desktop/Horse/horse/peds_ft.txt'
ll.learn_lgb()

  0%|          | 0/18638 [00:00<?, ?it/s]

  0%|          | 0/18638 [00:00<?, ?it/s]

pe finish
pe regularizrd


  0%|          | 0/276 [00:00<?, ?it/s]

  0%|          | 0/276 [00:00<?, ?it/s]

  0%|          | 0/276 [00:00<?, ?it/s]

C:\Users\Owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Total Bins 47243
[LightGBM] [Info] Number of data points in the train set: 96523, number of used features: 172


# grid search

In [5]:
train_data = ll.r.data_c
train, valid = split_data(train_data,test_size=0.2,rank_learning=False)
# 0.8 : 0.2

x_train = train.drop(['rank', 'date','単勝'], axis=1)
y_train = train['rank']
x_valid = valid.drop(['rank', 'date','単勝'], axis=1)
y_valid = valid['rank']
train_query = x_train.groupby(x_train.index).size()
train = lgb.Dataset(x_train, y_train, group=train_query)
valid_query = x_valid.groupby(x_valid.index).size()
valid = lgb.Dataset(x_valid, y_valid, reference=train, group=valid_query)


def score(params):
    print("Training start:")

    N_boost_round = []
    Score = []

    lgb_results={}  #履歴格納用

    lgb_clf = lgb.train(
        params,
        train,
        num_boost_round=1000,
        valid_sets=valid,
        valid_names=['valid'],
        early_stopping_rounds=20,
        verbose_eval=5,
        evals_result=lgb_results,
    )
#     return lgb_results
    return {'loss': -1.0 * lgb_results['valid']['ndcg@3'][lgb_clf.best_iteration], 'status': STATUS_OK}

def optimize(trials):
#探索スペース
    space = {
        'objective': 'lambdarank',
        'metric': 'ndcg',
        # 上位三着を考慮する
        'ndcg_eval_at': [1,2,3],
#         best paramsの返り値は, choiceだとindexか?
        'lambdarank_truncation_level': hp.choice('lambdarank_truncation_level',range(1,50)),
        'learning_rate': hp.uniform('learning_rate', 0.01, 0.1),
        # n_estimators == num_iterations
        'num_iterations': hp.choice('num_iterations',range(50,120)),
        'boosting_type': 'gbdt',
        'random_state': 777
    }

    max_evals = 25      #探索回数(25くらいで十分)
    best = fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=max_evals)

    print("best parameters:", best)


trials = Trials()
optimize(trials)


Training start:                                       
  0%|          | 0/25 [00:00<?, ?trial/s, best loss=?]

C:\Users\Owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")

C:\Users\Owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "

C:\Users\Owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will 

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.058385 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 47243                    
[LightGBM] [Info] Number of data points in the train set: 96523, number of used features: 172
Training until validation scores don't improve for 20 rounds
  0%|          | 0/25 [00:01<?, ?trial/s, best loss=?]

C:\Users\Owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')

C:\Users\Owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')



[5]	valid's ndcg@1: 0.540571	valid's ndcg@2: 0.52705	valid's ndcg@3: 0.517194
[10]	valid's ndcg@1: 0.535429	valid's ndcg@2: 0.524805	valid's ndcg@3: 0.515757
[15]	valid's ndcg@1: 0.542286	valid's ndcg@2: 0.526135	valid's ndcg@3: 0.517458
[20]	valid's ndcg@1: 0.537143	valid's ndcg@2: 0.524808	valid's ndcg@3: 0.517306
Early stopping, best iteration is:                    
[1]	valid's ndcg@1: 0.556	valid's ndcg@2: 0.536289	valid's ndcg@3: 0.522442
Training start:                                                                  
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.047858 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 47243                                               
[LightGBM] [Info] Number of data points in the train set: 96523, number of used features: 172
Training until validation scores don't improve for 20 rounds                     
[5]	valid's ndcg@1: 0.534857	valid's ndcg@2: 0

# milestone

In [ ]:
'lambdarank_truncation_level': 46, 'learning_rate': 0.023221403531782224, 'num_iterations': 33}

In [4]:
lgbm_params = {
                'metric': 'ndcg',
                'objective': 'lambdarank',
                'ndcg_eval_at': [1,2,3],
                'boosting_type': 'gbdt',
                'random_state': 777,
                'lambdarank_truncation_level': 47,
                'learning_rate':0.023221403531782224,
                'num_iterations': 83,
                # 'lambdarank_truncation_level': 10,
                # 'learning_rate': 0.02273417953255777,
                # n_estimators は num_iterations と同義　default = 100
                # 'n_estimators': 97,
                # 'num_leaves': 42,
                'force_col_wise':True
            }



In [8]:
import warnings

warnings.filterwarnings("ignore")

ll.learn_lgb2(train,lgbm_params=lgbm_params)

[LightGBM] [Info] Total Bins 47243
[LightGBM] [Info] Number of data points in the train set: 96523, number of used features: 172


# Simulate


In [5]:
# !pwd
save_path = '/Users/Owner/Desktop/Horse/ll_obj.pickle'

def save_pickle(save_path,object_):
    with open(save_path, mode="wb") as f:
        pickle.dump(object_, f)

def load_pickle(save_path):
    with open(save_path, mode="rb") as f:
        object_ = pickle.load(f)
    return object_

# save_pickle(save_path,ll)
ll = load_pickle(save_path)
ll.learn_model_ft()


In [36]:



for odds in [1]:
    print()
    sl = RankSimulater(ll.model)
    print("odds",odds)
    fukusho_df = sl.get_result_df(ll.r.data_c.loc[ll.x_test.index[0]:],return_tables,kaime='fukusho',odds=odds)
    tansho_df = sl.get_result_df(ll.r.data_c.loc[ll.x_test.index[0]:],return_tables,kaime='tansho',odds=odds)



odds 1


# calc results df

In [11]:
def calc_fukusho_acc(fukusho_df,odds=1.1,bet=100):
    race_num = len(fukusho_df)
    count_in3 = 0
    count_list = [0 for _ in range(3)]

    for i in range(len(fukusho_df)):
        col = fukusho_df.iloc[i]
        actual_rank_list = col['actual rank list']
        pred_list = col['pred list']
        fukusho_list = col['fukusho list']
        is_in = False
        visited = False

        for i,pred in enumerate(pred_list):
            is_in =  pred in fukusho_list
            if is_in and not visited:
                count_in3 += 1
                visited = True

            if pred_list[i] in fukusho_list:
                count_list[i] += 1
                
    # 予測した3頭の馬のうち, 1頭以上複勝圏に含まれている割合
    acc_in3 = count_in3/race_num * 100
    acc_list = [i/race_num * 100 for i in count_list]

    print("予測した3頭の馬のうち, 1頭以上複勝圏に含まれている割合",acc_in3)
    print("予測した3頭のそれぞれの複勝率",acc_list)
    

def calc_tansho_accuracy(result_df,bet=100):
    race_num = len(result_df)
    atari_num = result_df[result_df['is_atari']==True].sum()['is_atari']
    acc = (atari_num/race_num) * 100
    
    buy_num = result_df[result_df['is_buy']==True].sum()['is_buy']
    buy_df = result_df[result_df['is_buy']]
    atari_buy_num = buy_df[buy_df['is_atari']==True].sum()['is_atari']
    # 条件付き精度 (買ったもとで当たった確率)
    acc_cond = (atari_buy_num/buy_num) * 100
    total_expense = buy_num * bet
    return_price = result_df.sum()['profit']
    # 回収率
    profit_rate = (total_expense + return_price)/total_expense * 100
    atari_dist = {i:0 for i in range(1,13)}
    atari_buy_df = buy_df[buy_df['is_atari']==True]

    for race_id in atari_buy_df.index:
        race_num = int(str(race_id)[-2:])
        atari_dist[race_num] += (1/atari_buy_num) * 100
    
    print('acc ',acc)
    print('acc cond',acc_cond)
    print('profit rate',profit_rate)
    print('atari race distribution',atari_dist)



In [37]:
calc_fukusho_acc(fukusho_df)

予測した3頭の馬のうち, 1頭以上複勝圏に含まれている割合 86.20302860347728
予測した3頭のそれぞれの複勝率 [50.1402131239484, 41.61525518788559, 38.75490745933819]


In [39]:

calc_tansho_accuracy(tansho_df)


acc  22.994952327537856
acc cond 22.87323943661972
profit rate 104.82816901408452
atari race distribution {1: 10.344827586206895, 2: 8.866995073891625, 3: 8.374384236453201, 4: 9.60591133004926, 5: 8.620689655172413, 6: 8.12807881773399, 7: 10.591133004926107, 8: 7.881773399014777, 9: 7.635467980295565, 10: 6.403940886699506, 11: 5.1724137931034475, 12: 8.374384236453201}


# race_id_list

In [5]:

# race_id_list = ['2022040303{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022100403{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022010203{}'.format(str(i).zfill(2)) for i in range(1,13)]

race_id_list = ['2022040304{}'.format(str(i).zfill(2)) for i in range(1,13)]
race_id_list += ['2022100404{}'.format(str(i).zfill(2)) for i in range(1,13)]
race_id_list += ['2022010204{}'.format(str(i).zfill(2)) for i in range(1,13)]



# race_id_list += ['2022050308{}'.format(str(i).zfill(2)) for i in range(1,13)]

# race_id_list += ['2022090301{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022090302{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022090303{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022090304{}'.format(str(i).zfill(2)) for i in range(1,13)]

# race_id_list += ['2022020103{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022020104{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022020105{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022020106{}'.format(str(i).zfill(2)) for i in range(1,13)]



In [48]:
race_id_list = []

for place in range(1, 11, 1):
    for kai in range(1, 6, 1):
        for day in range(1, 13, 1):
            for r in range(1, 13, 1):
                race_id = "2022" + str(place).zfill(2) + str(kai).zfill(2) +\
		str(day).zfill(2) + str(r).zfill(2)
                race_id_list.append(race_id)

# Predictor

In [7]:
pt = Predictor(peds,results,horse_results,race_id_list)
# ********* test_size = 0  : ずっと0にしてなかった...
pt.path_ft = '/Users/Owner/Desktop/Horse/horse/peds_ft.txt'
pt.learn_lgb(lgbm_params,test_size=0)

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/18673 [00:00<?, ?it/s]

  0%|          | 0/18673 [00:00<?, ?it/s]

...

  0%|          | 0/276 [00:00<?, ?it/s]

  0%|          | 0/276 [00:00<?, ?it/s]

  0%|          | 0/276 [00:00<?, ?it/s]

C:\Users\Owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Total Bins 49464
[LightGBM] [Info] Number of data points in the train set: 120314, number of used features: 172


# 新潟

In [ ]:
base  = '2022040303'
pt.predict(base+'12')

# 小倉

In [ ]:
base  = '2022100403'
# for i in ['07','08','09','10','11']:
pt.predict(base+'12')

# 札幌

In [ ]:
base  = '2022010203'
# for i in ['07','08','09','10','11']:
pt.predict(base+'12')

# 当日の収支計算

return_tables　と, return_tables_todayで形式が異なるので, 形式をそろえる

# 当日 sim 手順

In [35]:
fukusho_row = return_table[return_table[0]=='複勝']
fukusho_odds_list = fukusho_row[2].str.split('円').values[0][0:3]
fukusho_odds_list = [i for i in fukusho_odds_list if i!='']
fukusho_odds = list(map(lambda x: int(x.replace(',',''))/100 , fukusho_odds_list))

In [36]:
fukusho_odds

[1.1, 1.1]

In [34]:
fukusho_odds_list = fukusho_row[2].str.split('円').values[0][0:3]
fukusho_odds_list.remove('')
fukusho_odds_list

['110', '110']

In [32]:
len(fukusho_row[2].str.split('円').values[0][0:3])

3

In [23]:


return_tables_ = tsl.return_tables
indexes = set(return_tables_.index)

for idx in indexes:
    return_table = return_tables_.loc[idx]
    fukusho_row = return_table[return_table[0]=='複勝']
    print(fukusho_row)


               0        1             2          3
202204030404  複勝  11 9 13  110円220円140円  1人気4人気3人気
               0      1             2          3
202201020411  複勝  4 3 9  180円170円310円  3人気2人気5人気
               0      1             2          3
202210040412  複勝  1 3 9  110円130円150円  1人気2人気4人気
               0        1             2          3
202210040404  複勝  17 12 5  130円150円240円  1人気3人気4人気
               0       1             2          3
202201020403  複勝  1 6 10  190円110円170円  4人気1人気2人気
               0       1             2          3
202201020407  複勝  6 12 3  200円230円150円  3人気4人気1人気
               0       1             2          3
202204030405  複勝  9 4 11  150円120円310円  2人気1人気6人気
               0       1             2          3
202210040408  複勝  14 7 6  170円600円490円  1人気9人気6人気
               0      1               2          3
202201020409  複勝  4 6 2  1,100円150円230円  8人気1人気5人気
               0      1             2          3
202210040401  複勝  4 2 3  110円290円250円  1人気5人気3人気


In [40]:
data =  ShutubaTable.scrape(race_id_list, pt.date)
st = ShutubaTable(data)
st.preprocessing()
st.merge_horse_results(pt.hr)
st.merge_peds(pt.pe.peds_vec)
st.process_categorical(pt.r.le_horse, pt.r.le_jockey, pt.r.data_pe)


tsl = TodaySimulater(pt.model)
tsl.return_table_today(race_id_list)
all_results = tsl.get_result_df( st.data_c, tsl.return_tables,race_id_list, kaime='all',odds=5)

  0%|          | 0/36 [00:00<?, ?it/s]

In [41]:
all_results


,pred_list,actual_rank_list,tansho_odds,fukusho_odds,umaren_odds,wide_odds,umatan_odds,sanrenpuku_odds,sanrentan_odds,wide_comb
202204030401,"[7, 6, 10, 2, 3, 8, 5, 4, 9, 1]","[10, 6, 7]",8.1,"[1.4, 1.0, 1.6]",4.2,"[1.9, 4.3, 2.1]",15.4,7.4,59.4,"[[6, 10], [7, 10], [6, 7]]"
202204030402,"[9, 6, 7, 5, 3, 8, 2, 4, 1]","[7, 4, 9]",3.1,"[1.4, 23.0, 1.8]",295.5,"[70.8, 3.2, 83.5]",396.6,295.4,2036.3,"[[4, 7], [7, 9], [4, 9]]"
202204030403,"[7, 4, 12, 1, 6, 8, 11, 17, 2, 14, 13, 15, 16,...","[12, 8, 6]",5.0,"[2.1, 5.0, 2.7]",40.9,"[12.0, 6.7, 23.9]",69.2,91.4,402.0,"[[8, 12], [6, 12], [6, 8]]"
202204030404,"[13, 11, 5, 9, 12, 2, 10, 4, 6, 1, 15, 3, 14, ...","[11, 9, 13]",2.3,"[1.1, 2.2, 1.4]",12.6,"[4.8, 3.0, 5.1]",15.5,16.0,65.9,"[[9, 11], [11, 13], [9, 13]]"
202204030405,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]","[9, 4, 11]",4.0,"[1.5, 1.2, 3.1]",5.5,"[2.7, 8.0, 7.3]",10.8,19.1,84.6,"[[4, 9], [9, 11], [4, 11]]"
202204030406,"[6, 12, 14, 8, 15, 9, 13, 4, 16, 11, 2, 5, 18,...","[8, 6, 14]",2.8,"[1.2, 1.5, 1.8]",5.9,"[2.4, 4.9, 4.7]",10.8,13.8,51.7,"[[6, 8], [8, 14], [6, 14]]"
202204030407,"[2, 8, 4, 3, 7, 10, 12, 14, 6, 15, 9, 13, 1, 5...","[2, 3, 10]",2.7,"[1.4, 3.4, 5.5]",14.7,"[7.3, 14.8, 58.0]",21.7,146.0,478.1,"[[2, 3], [2, 10], [3, 10]]"
202204030408,"[8, 7, 9, 11, 12, 4, 2, 10, 1, 3, 6, 5]","[11, 12, 4]",7.2,"[4.5, 6.6, 23.7]",27.6,"[6.9, 23.0, 32.1]",66.1,313.2,1447.5,"[[11, 12], [4, 11], [4, 12]]"
202204030409,"[8, 12, 3, 2, 10, 6, 17, 7, 11, 1, 15, 13, 18,...","[12, 17, 18]",11.3,"[3.8, 2.1, 2.5]",21.9,"[7.5, 10.7, 6.9]",58.6,52.9,349.4,"[[12, 17], [12, 18], [17, 18]]"
202204030410,"[6, 5, 4, 7, 2, 1, 3, 9, 8]","[5, 2, 1]",1.8,"[1.1, 1.7, 1.7]",6.1,"[2.7, 3.4, 6.2]",7.4,12.3,33.8,"[[2, 5], [1, 5], [1, 2]]"


# 日付に注意

In [ ]:
date = '2022/12/31'

# Results scraping

In [65]:
# race_id_list = results.index.astype('str')

results_tmp = Results.scrape(race_id_list)

  0%|          | 0/5568 [00:00<?, ?it/s]

# Horse_results scraping

In [66]:
horse_id_list = results_tmp['horse_id'].astype(str).unique()
horse_results_tmp = HorseResults.scrape(horse_id_list)

# save_path = '/Users/rince/Desktop/Horse/Data/horse_2020.csv'

  0%|          | 0/4712 [00:00<?, ?it/s]

# Peds scraping

In [67]:
peds_2021 = Peds.scrape(horse_id_list)
pe_2021 = Peds(peds_2021)
pe_2021.regularize_peds()
peds_tmp = pe_2021.peds_re

  0%|          | 0/4712 [00:00<?, ?it/s]

  0%|          | 0/4712 [00:00<?, ?it/s]

# Return scraping

In [68]:
returns_tmp = Return.scrape(race_id_list)


  0%|          | 0/5568 [00:00<?, ?it/s]

# update

In [69]:
new_results = update_data(load_csv(path_win+'results.csv'),results_tmp)
new_horse_results = update_data(load_csv(path_win+'horse_results.csv'),horse_results_tmp)
new_peds = update_data(load_csv(path_win+'peds.csv'),peds_tmp)
returns = load_csv(path_win+'return.csv')
returns.rename(columns={'0':0,'1':1,'2':2,'3':3},inplace=True)
returns_tmp.rename(columns={'0':0,'1':1,'2':2,'3':3},inplace=True)
new_return = update_data(returns,returns_tmp)


# save

In [70]:
new_results.to_csv(path_win+'results.csv')
new_horse_results.to_csv(path_win+'horse_results.csv')
new_peds.to_csv(path_win+'peds.csv')
new_return.to_csv(path_win+'return.csv')

# 重要度

In [ ]:
importances = pd.DataFrame(
{'features' : x_train.columns, 'importances' : lgb_rank.feature_importance()})
print(importances.sort_values('importances', ascending=False)[:30])

# fastText

流れ
1. fasttext用の血統データの学習データを作る (血統の情報のみ, index ヘッダはいらない)
2. fasttext学習
3. 学習モデルを使って, 血統データをベクトル化
4. ベクトル化して r.data_cに concat
5. 学習


教師あり, 教師なしでも生成されるベクトルは等しい

# model_ft 作成

In [ ]:
# 相対パスしかできない
# dim : 出力の次元
# minn : n_gramの最小単位
# maxn : n_gramの最大単位
path_ft = '/Users/rince/Desktop/Horse/code/horse/peds_ft.txt'
# 上書き保存OK
peds.to_csv(path_ft,header=False,index=False,sep=',')
model_ft = ft.train_unsupervised(path_ft,dim=62,minn=2,maxn=14)

model[model.words[1]] と model.get_input_vector(ind=1) は等価

In [ ]:
lis_ = [1,2,3,4,True,4]
lis2_ = [2,3,4,5,False,5]
dic_ = {
}
dic_['a'] = lis_
dic_['b'] = lis2_

df_ = pd.DataFrame(dic_).T

In [ ]:
df_

In [ ]:
df_.rename(columns={0:'profit',1:'is_atari',2:'is_buy',3:'actual_rank',4:'not_buy_reason',5:'is_error'})